# Export XML to TSV

In [1]:
import csv
import pathlib
import xml.etree.ElementTree
import lzma
import re

In [2]:
pattern = re.compile(r'https://w3id\.org/oc/corpus/br/([0-9]+)')

def br_uri_to_id(uri):
    """
    https://w3id.org/oc/corpus/br/922634 to 922634
    """
    match = pattern.search(uri)
    return match.group(1)

In [3]:
xml_path = pathlib.Path('data/cites.xml')
tsv_path = pathlib.Path('data/cites.tsv.xz')

with xml_path.open() as read_file, lzma.open(tsv_path, 'wt') as write_file:
    parser = xml.etree.ElementTree.iterparse(read_file)
    writer = csv.writer(write_file, delimiter='\t')
    writer.writerow(('source_uri', 'target_uri'))
    ns = '{http://www.w3.org/2005/sparql-results#}'
    for event, elem in parser:
        assert event == 'end'
        if elem.tag == f'{ns}result':
            source = elem.findtext(f"{ns}binding[@name='source']/{ns}uri")
            target = elem.findtext(f"{ns}binding[@name='target']/{ns}uri")
            source = br_uri_to_id(source)
            target = br_uri_to_id(target)
            writer.writerow((source, target))
            elem.clear()

ParseError: no element found: line 8951905, column 26 (<string>)